# Self-Supervised Material and Texture Representation Learning for Remote Sensing Tasks

In [4]:
# Standard packages
import csv
import h5py
import os
import time
import math

# Math packages
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
# PyMca packages
from PyMca5.PyMca import ConfigDict

# XRF image processing packages
import xrfip as xp
import xrfip.optimization as opt
from xrfip.optimization.losses import *

from xrfip.utils.errors import *
from xrfip.utils.units import *
from xrfip.utils.visualizations import *
from xrfip.utils.save_mask import save_mask


from einops.layers.torch import Rearrange



import pystac
from pystac.client import Client
from IPython.display import display, Image

/opt/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /opt/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <A54E39C4-6B62-3303-9BE6-7DB88EB078BF> /opt/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
#ResNet-34 Model
class ResNet(nn.Module):
    expansion = 1
    def __init__(self, channels):
        super(ResNet,self).__init__()
        self.inplanes = 64
        self.num_clusters = 64
        self.in_conv = torch.nn.Conv2d(in_channels = channels, out_channels = 64, kernel_size = 7, padding = 3)
        self.layers = make_layers(layer = ResBlock, num_layers = 4, in_channels = [64,128,256,512], out_channels = [128,256,512,channels], kernel_size = 3, padding = 1)
        self.out_conv = torch.nn.Conv2d(in_channels = channels, out_channels = channels, kernel_size= 3, padding = 1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu =nn.ReLU(inplace = True)
        self.softmax = nn.Softmax2d()
        self.relu = nn.ReLU(inplace = True)
        self.bn2 = nn.BatchNorm2d(channels)
        self.softplus = nn.Softplus()
    def make_layers(layer, num_layers,inplanes, planes, strides,  kernel_size, padding):
        strides = [strides] + [1] * (num_layers-1)
        layers = []
        for i in range(num_layers):
            stride = strides[i]
            layers.append(ResBlock(inplanes), planes, stride)
            inplanes = planes * i
    def forward(self, x):
        x = self.in_conv(x)
        x = self.bn1(x)  
        x = self.relu(x)
        x = self.layers(x)
        x = self.softmax(x)
        x = self.out_conv(x)
        x = self.bn2(x)
        x = self.softplus(x)
        return x


            
#Resblock is done, which operates as one block of ResNet with adding x and f(x)
class ResBlock(nn.Module):
    #we use expansion here to generate the array of expansion
    expansion = 1
    #initalization
    def __init__(self, in_planes, planes, stride = 1, is_last = False):
        super(ResBlock, self).init__()  
        #standard convolutional block
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.shortcut = nn.Sequential()
        #stride != 1, that means we can take the shortcut because we are not at an inital block
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )
    def forward(self, x):
        out = torch.nn.functional(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(out)
        pre = out
        out = nn.ReLU(out)
        if self.is_last:
            return out, pre
        else:
            pre
        
            
            

In [8]:
class LowLevelBlock(nn.Module):
    def __init__(self, in_channels =3 , out_channels= 64):
        super(LowLevelBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels =in_channels, out_channels= out_channels, kernel_size= 3, stride = 1, padding = 1)
        self.act1 = nn.LeakyReLU(0.1, inplace=False)
        self.conv2 = nn.Conv2d(in_channels= out_channels, out_channels = out_channels, kernel_size=3, stride = 1, padding = 1)
        self.act2 = nn.LeakyReLU(0.1, inplace = False)
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.act2(self.conv2(x))
        
class RefineBlock(nn.Module):
    def __init__(self, channels = 64):
        super(RefineBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = channels, out_channels= channels, kernel_size=3, stride = 1, padding = 1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(in_channels = channels, out_channels= channels, kernel_size= 3, stride = 1, padding = 1)
        self.bn2 =nn.BatchNorm2d(channels)
        self.relu = nn.ReLU(inplace = True)
    def forward(self, x):
        res = x 
        out = self.relu(self.BatchNorm2d(x))
        
#Texture Refinement Network (TeRn)
class EvalNet(nn.Module):
    """φ_e: takes a token grid S (with masks) and predicts a keep/mask probability map."""
    def __init__(self, dimensions, n_heads, depth):
        super(EvalNet, self).__init__()
        layers += [
            nn.MultiheadAttention(dimensions, n_heads, batch_first= True),
            nn.LayerNorm(dimensions),
            nn.Linear(dimensions),
            nn.GELU(),
            nn.LayerNorm(dimensions)
        ]
        self.net = nn.Sequential(*layers)
        self.to_logits = nn.Linear(dim = 1) #uses one logit per token
    
    def forward(self, tokens):
        x = self.net(tokens)
        logits = self.to_logits(x)
        token = torch.sigmoid(logits)
        return token

class RefineNet(nn.Module):
    def __init__(self, dim, n_heads):
        super(RefineNet, self).__init__()
        layers += [
            nn.MultiheadAttention(dim, n_heads, batch_first = True),
            nn.LayerNorm(dim),
            nn.Linear(dim),
            nn.GELU(),
            nn.LayerNorm(dim)
        ]
        self.net = nn.Sequential(*layers)
        self.to_logits = nn.Linear(dim =  1)
    def forward(self, token):
        
        

IndentationError: expected an indented block after class definition on line 11 (949958565.py, line 13)

In [ ]:
from .alias_multinomial import AliasMultinomial



In [7]:
#Encoder
class TextureEncoder(nn.Module):
    def __init__(self, channels, num_clusters):
        super(TextureEncoder, self).__init__()
        self.inchannels, self.num_clusters = channels, num_clusters
        std = 1. / ((num_clusters * channels)**0.5)
        self.clusters = torch.zeros(num_clusters, channels, dtype = torch.float, requires_grad = True).uniform(-std, std)
        self.scale =torch.zeros(num_clusters, dtype = torch.float, requires_grad= True).uniform_(-1,0)
        
    def l2_norm(self, x, clusters, scale) ->torch.tensor:
        """ using l2 distance we find the weights of cluster centers which is needed for such feature extraction
        args:
        x: our input features
        clsuters; learning clusters
        scale: we scale by some term
        """
        num_clusters, in_channels = clusters.size()
        batch_size = x.size(0)
        scale = scale.view((1,1, clusters)) 
        
        
        x  = x.unsqueeze(2).expand(batch_size, x.size(1), num_clusters, in_channels)
        clusters = clusters.view(1,1,num_clusters,in_channels)
        
        norm = scale * (x - clusters).pow(2).sum(dim=3)
        
        return norm
    def gather(self, theta, x , clusters)->torch.tensor:
        """gather up all the clusters together into a tensor
        args
        x : input features
        theta : learned cluster weights
        clusters :  learned clusters
        
        returns the accumlated tensor"""
        
        num_clusters, channels = clusters.size()
        clusters = clusters.view((1,1,  num_clusters, channels))
        batch_size = x.size(0)
        
        x = x.unsqueeze(2).expand((batch_size, x.size(1), num_clusters, channels))
        residual = x - clusters
        encoded_feat = (theta.unsqueeze(3)*residual).sum(dim=1)
        
        return encoded_feat
        
    def forward(self, x):
        """forward function for  surface encoding
        Args
        x: input features
        
        returns the residual features"""
        assert x.dim() == 4  and x.size(1) == self.inchannels, x.shape
        x = Rearrange(x , 'b c h w -> b (h w) c').contiguous()
        theta = nn.softmax(self.l2_norm(x, self.clusters, self.scale), dim = 2)
        #accumelate
        encoded_feat = self.gather(theta, x, self.clusters)
        
        return encoded_feat



In [ ]:
from math import isclose

class AliasMultinomial(nn.Module):
    def __init___(self, prob):
        super(AliasMultinomial, self).__init__()
        assert isclose(prob.sum().item(), 1), 'nosie distrubtion 1'
        cpu_prob = prob.cpu()
        K = len(prob)
        
        self.prob = [0] * K
        self.alias = [0] * K 
        
        small = []
        large = []
        for index, prob in enumerate(cpu_prob):
            self.prob[index] = K* prob
            if self.prob[index] < 1.0:
                small.append(index)
            else: 
                large.append(index)
        while len(small) > 0 and len(large) > 0: 
            small = small.pop(
                
            )

In [ ]:
catalog = Client.open("https://earth-search.aws.element84.com/v1")

# 2. Do the same search as before
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=[-122.6, 37.6, -122.3, 37.9],
    datetime="2021-08-01/2021-08-31",
    query={"eo:cloud_cover": {"lt": 10}},
    limit=10
)

items = list(search.get_items())

# 3. Display each item’s thumbnail
for item in items:
    thumb_href = item.assets["thumbnail"].href  # usually a small JPEG/PNG
    print(item.id, item.datetime)
    display(Image(url=thumb_href))
